In [23]:
import os
import pandas as pd
import re

In [24]:
# tcai - all content has been merged to get_data.py

mobility_path = '../data/raw/'
processed_path = '../data/processed/'
raw_path = '../data/raw/'


In [25]:
# mobility_combine = []

# for file in os.listdir(mobility_path):
#     df = pd.read_csv(path + file, sep='\t')
#     df = df.loc[(df['Kind'] == 'subregion')]
#     mobility_combine.append(df)

# mobility_df_combine = pd.concat(mobility_combine)

# mobility_df_combine.to_csv(path+'mobility_county.csv')

In [26]:
mobility = pd.read_csv(mobility_path+'Global_Mobility_Report.csv', low_memory=False)
processed = pd.read_csv(processed_path + 'time_series.csv')
zips = pd.read_csv(raw_path + 'uszips.csv')

In [27]:
# clean USA data
USA = mobility.loc[mobility['country_region'] == 'United States'].reset_index()
to_drop = ['index', 'country_region_code', 'country_region']
USA = USA.drop(to_drop, axis=1)
USA.insert(1, 'county', USA['sub_region_2'].str.lower())
USA = USA.drop(['sub_region_2'],axis=1)
USA = USA.loc[~pd.isna(USA['county'])]
USA['county'] = USA['county'].map(lambda x: re.sub(' county| city','', x)).str.lower()
USA.insert(1, 'state', USA['sub_region_1'].str.lower())
USA = USA.drop(['sub_region_1'],axis=1)

In [28]:
USA

,state,county,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
114,alabama,autauga,2020-02-15,5.0,7.0,NaN,NaN,-4.0,NaN
115,alabama,autauga,2020-02-16,0.0,1.0,-23.0,NaN,-4.0,NaN
116,alabama,autauga,2020-02-17,8.0,0.0,NaN,NaN,-27.0,5.0
117,alabama,autauga,2020-02-18,-2.0,0.0,NaN,NaN,2.0,0.0
118,alabama,autauga,2020-02-19,-2.0,0.0,NaN,NaN,2.0,0.0
...,...,...,...,...,...,...,...,...,...
155055,wyoming,weston,2020-04-06,NaN,NaN,NaN,NaN,-39.0,NaN
155056,wyoming,weston,2020-04-07,NaN,NaN,NaN,NaN,-37.0,NaN
155057,wyoming,weston,2020-04-08,NaN,NaN,NaN,NaN,-36.0,NaN
155058,wyoming,weston,2020-04-09,NaN,NaN,NaN,NaN,-31.0,NaN


In [29]:
processed['county'] = processed['county'].str.lower()
processed['state'] = processed['state'].str.lower()

In [30]:
processed

,Unnamed: 0,date,county,state,fips,cases,deaths,county_fips_all,timestamp,cldCvrMin,...,windSpdMax,windSpd80mMin,windSpd80mAvg,windSpd80mMax,windSpd100mMin,windSpd100mAvg,windSpd100mMax,wetBulbMin,wetBulbAvg,wetBulbMax
0,0,2020-01-21,snohomish,washington,53061,1,0,53061,2020-01-21,7.000000,...,9.900000,14.800000,18.9,22.10,14.000000,18.0,20.7,41.60,42.70,44.1
1,1,2020-01-22,snohomish,washington,53061,1,0,53061,2020-01-22,31.000000,...,8.700000,9.600000,13.6,21.20,10.000000,13.2,20.5,41.40,42.40,44.6
2,2,2020-01-23,snohomish,washington,53061,1,0,53061,2020-01-23,75.000000,...,9.200000,11.000000,15.5,21.40,10.600000,15.4,19.4,44.90,48.60,50.7
3,3,2020-01-24,cook,illinois,17031,1,0,17031,2020-01-24,96.333333,...,7.933333,1.933333,8.0,14.00,1.766667,8.4,16.0,32.30,34.60,37.0
4,4,2020-01-24,snohomish,washington,53061,1,0,53061,2020-01-24,15.000000,...,11.200000,10.800000,15.3,23.20,9.600000,14.2,20.7,44.70,46.70,48.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67373,67373,2020-04-22,oconto,wisconsin,55083,5,0,55083,2020-04-22,0.000000,...,12.200000,3.000000,12.1,17.15,4.250000,12.3,16.6,24.25,29.45,34.3
67374,67374,2020-04-22,rusk,wisconsin,55107,4,0,55107,2020-04-22,0.000000,...,12.600000,12.800000,15.8,18.40,10.000000,15.8,19.1,27.60,36.00,44.7
67375,67375,2020-04-22,shawano,wisconsin,55115,6,0,55115,2020-04-22,0.000000,...,15.700000,3.600000,12.8,18.10,4.800000,12.7,17.4,25.30,30.70,35.3
67376,67376,2020-04-22,campbell,wyoming,56005,14,0,56005,2020-04-22,0.000000,...,14.300000,4.300000,15.2,24.80,4.200000,15.5,26.5,36.70,43.10,49.3


In [31]:
# take only subset of counties according to proccessed data
counties = pd.unique(processed['county'].str.lower().map(lambda x: re.sub(' city','', x)))
USA = USA.loc[USA['county'].str.lower().isin(counties)]

In [32]:
# difference in counties
difference = set(pd.unique(counties)) - set(USA['county'].str.lower().unique())

In [10]:
# Cut time
# USA = USA.loc[USA['date']<'2020-04-23']

In [33]:
# deal with duplicates
columns = set(USA.columns) - set(['state', 'county', 'date'])
agg = {key:'mean' for key in columns}
USA_grouped = USA.groupby(['state', 'county', 'date']).aggregate(agg)

In [75]:
df = pd.merge(processed,USA_grouped, on=['date','state', 'county'],how='left')

In [69]:
len(df[['date', 'county', 'county_fips_all', 'timestamp']].drop_duplicates())-len(df)

0

In [79]:
df

,Unnamed: 0,date,county,state,fips,cases,deaths,county_fips_all,timestamp,cldCvrMin,...,windSpd100mMax,wetBulbMin,wetBulbAvg,wetBulbMax,retail_and_recreation_percent_change_from_baseline,residential_percent_change_from_baseline,transit_stations_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,workplaces_percent_change_from_baseline,parks_percent_change_from_baseline
0,0,2020-01-21,snohomish,washington,53061,1,0,53061,2020-01-21,7.000000,...,20.7,41.60,42.70,44.1,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2020-01-22,snohomish,washington,53061,1,0,53061,2020-01-22,31.000000,...,20.5,41.40,42.40,44.6,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2020-01-23,snohomish,washington,53061,1,0,53061,2020-01-23,75.000000,...,19.4,44.90,48.60,50.7,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2020-01-24,cook,illinois,17031,1,0,17031,2020-01-24,96.333333,...,16.0,32.30,34.60,37.0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2020-01-24,snohomish,washington,53061,1,0,53061,2020-01-24,15.000000,...,20.7,44.70,46.70,48.3,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67373,67373,2020-04-22,oconto,wisconsin,55083,5,0,55083,2020-04-22,0.000000,...,16.6,24.25,29.45,34.3,NaN,NaN,NaN,NaN,NaN,NaN
67374,67374,2020-04-22,rusk,wisconsin,55107,4,0,55107,2020-04-22,0.000000,...,19.1,27.60,36.00,44.7,NaN,NaN,NaN,NaN,NaN,NaN
67375,67375,2020-04-22,shawano,wisconsin,55115,6,0,55115,2020-04-22,0.000000,...,17.4,25.30,30.70,35.3,NaN,NaN,NaN,NaN,NaN,NaN
67376,67376,2020-04-22,campbell,wyoming,56005,14,0,56005,2020-04-22,0.000000,...,26.5,36.70,43.10,49.3,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
df.to_csv(processed_path + 'time_series_mobility.csv')
# df = pd.read_csv(processed_path + 'time_series_mobility.csv')